In [ ]:
# Install necessary Python pakcages
# rdflib: RDF composition and querying with SPARQL in Python.
from rdflib import RDFS, RDF, Namespace, Graph, URIRef, Literal

from common import *

In [ ]:
import json
import pandas as pd
import re

### Sample raw metadata

1. Those data are fairly semi-structured by some conventions.
  - Each information is somewhat separated by dots.

In [ ]:
raw_metadata_dict = {
    '51': 'BLDA.RM-2150..ZN-T',
    '43': 'BLDA.RM-2150..DMPR-POS',
    '1':  'BLDA.AH1.CHWR1-T'
}

### Regular expressions (RE)
1. We will use regular expressions to identify if certain information is included in the metadata.
2. Assuming that each information is separated by dots.

In [ ]:
# key: RE pattern, value: identified TagSet.
re_dict = {
    'BLDA': 'Building',
    'AH': 'AHU',
    'CHWR\\d-T': 'Chilled_Water_Return_Temperature_Sensor',
    'RM-\\d+': 'HVAC_Zone',
    'DMPR-POS': 'Damper_Position_Sensor',
    'ZN-T': 'Zone_Temperature_Sensor'
    
}

### VAV <-> Zone
1. VAV information is omitted in the metadata. From the prior knowledge, a VAV corresponds to a Zone.

### Zone <-> Room information
1. It is usually not given in metadata accessible through BMSes.
2. You need to make such information from any source you have (asking building managers or extract it from a schematic.)
3. Assume that it's given as below.

In [ ]:
zone_room_map = {
    "RM-2150": ["RM-2151", "RM-2152"]
}

### AHU <-> VAV information.
1. It is also not given by BMSes usually.
2. Assume given as below

In [ ]:
feeds_map = {
    'AH1': 'VAV_RM-2150'
}

### Init an RDF whiteboard.

In [ ]:
g = Graph() # Initialize a graph
RDFS # predefined namespace as 'http://www.w3.org/2000/01/rdf-schema#'
RDF # predefined namespace as 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
BRICK = Namespace('https://brickschema.org/schema/1.0.1/Brick#')
BF = Namespace('https://brickschema.org/schema/1.0.1/BrickFrame#')
EX = Namespace('http://example.com#')
g.bind('ex', EX)
g.bind('brick', BRICK)
g.bind('bf', BF)
g.bind('rdfs', RDFS)
g.bind('rdf', RDF)



### Let's convert into Brick.
- We will convert the first metadata first to see each step and then the others in a loop.

In [ ]:
# Initialization: Words separation
vendor_name = raw_metadata_dict['51']
entity_dict = dict() # This will contain all entities found in this vendor_name
raw_words = vendor_name.split('.') # This building uses '.' as a delimiter but not always it's comprehensive.
print('Raw metadata: {0}\nSeparated raw words: {1}'.format(vendor_name, raw_words))

In [ ]:
# Find entities in the words
entity_dict = dict()
for word in raw_words:
    for re_rule, tagset in re_dict.items():
        if re.findall(re_rule, word):
            entity_dict[word] = tagset

In [ ]:
# Define instance relationships
for entity, tagset in entity_dict.items():
    g.add((EX[entity], RDF['type'], BRICK[tagset]))
print_graph(g)

In [ ]:
# Assumtion that Zone is associated with VAV
for entity, tagset in entity_dict.items():
    if tagset == 'HVAC_Zone':
        vav_name = 'VAV_' + entity
        g.add((EX[vav_name], RDF['type'], BRICK['VAV']))
        g.add((EX[vav_name], BF['feeds'], EX[entity]))
print_graph(g)

In [ ]:
# Assumtion that zone and ZNT has location relationship
for entity, tagset in entity_dict.items():
    if tagset == 'Zone_Temperature_Sensor':
        for entity2, tagset2 in entity_dict.items():
            if tagset2 == 'HVAC_Zone':
                g.add((EX[entity], BF['isLocatedIn'], EX[entity2]))
print_graph(g)

### Apply the rules to the entire raw metadata

Refer the full versoin of Brick Conversion for actual iteration.